In [139]:
import os
import json
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from PIL import Image
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms.v2 as tfs

In [132]:
class DogDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform
        with open(os.path.join(self.path, "format.json"), "r") as file:
            self.format = json.load(file)
            
        self.len = 0
        self.all_files = []
        self.targets = torch.eye(10)
        
        for path_, target_ in self.format.items():
            full_path = os.path.join(self.path, path_)
            files = os.listdir(full_path)
            self.len += len(files)
            self.all_files.extend(map(lambda _path: (os.path.join(full_path, _path), target_), files))

    def __getitem__(self, item):
        img, target = self.all_files[item]
        target = self.targets[target]
        img = Image.open(img).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, target

    def __len__(self):
        return self.len

In [127]:
transform = models.ResNet50_Weights.DEFAULT.transforms()
d_train = DogDataset("dogs", train=True, transform=transform)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(models.ResNet50_Weights.DEFAULT)
model.requires_grad_(False)
model.fc = nn.Sequential(
    nn.Linear(512*4, 10)
)
model.fc.requires_grad_(True)
model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.fc.parameters(), lr=0.01, weight_decay=0.001)
eph = 15
for _e in range(eph):
    loss_mean, lm_count = 0, 0
    tqdm_train_data = tqdm(train_data, leave=True)
    for x_train, y_train in tqdm_train_data:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

        tqdm_train_data.set_description(f"Текущая эпоха: [{_e+1}/{eph}]  Текущая средняя ошибка: {round(loss_mean, 4)}")

Текущая эпоха: [1/15]  Текущая средняя ошибка: 0.6256: 100%|███████████████████████████| 51/51 [00:09<00:00,  5.21it/s]
Текущая эпоха: [2/15]  Текущая средняя ошибка: 0.1858: 100%|███████████████████████████| 51/51 [00:09<00:00,  5.17it/s]
Текущая эпоха: [3/15]  Текущая средняя ошибка: 0.1293: 100%|███████████████████████████| 51/51 [00:10<00:00,  5.10it/s]
Текущая эпоха: [4/15]  Текущая средняя ошибка: 0.1169: 100%|███████████████████████████| 51/51 [00:09<00:00,  5.19it/s]
Текущая эпоха: [5/15]  Текущая средняя ошибка: 0.0924: 100%|███████████████████████████| 51/51 [00:18<00:00,  2.68it/s]
Текущая эпоха: [6/15]  Текущая средняя ошибка: 0.0516: 100%|███████████████████████████| 51/51 [00:10<00:00,  5.07it/s]
Текущая эпоха: [7/15]  Текущая средняя ошибка: 0.0575: 100%|███████████████████████████| 51/51 [00:09<00:00,  5.32it/s]
Текущая эпоха: [8/15]  Текущая средняя ошибка: 0.0793: 100%|███████████████████████████| 51/51 [00:09<00:00,  5.30it/s]
Текущая эпоха: [9/15]  Текущая средняя о

In [128]:
#Вычислим долю верных классификай для тренировочной выборке
model.eval()
model.to(device)
d_train = DogDataset("dogs", train=True, transform=transform)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)
Q, cnt = 0, 0
with torch.no_grad():
    tqdm_train_data = tqdm(train_data, leave=True)
    for x_train, y_train in tqdm_train_data:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = torch.argmax(model(x_train).softmax(dim=1), dim=1).squeeze()
        Q += (torch.mean((predict == torch.argmax(y_train, dim=1).squeeze()).float())).item()
        cnt += 1
Q /= cnt
print(f"Процент верных классификации на тренировочной выборке: {round(Q*100, 3)}%")

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:09<00:00,  5.27it/s]

Процент верных классификации на тренировочной выборке: 99.632%


In [129]:
model.eval()
model.to(device)
d_test = DogDataset("dogs", train=False, transform=transform)
test_data = data.DataLoader(d_test, batch_size=10, shuffle=True)
Q, cnt = 0, 0
with torch.no_grad():
    tqdm_test_data = tqdm(test_data, leave=True)
    for x_test, y_test in tqdm_test_data:
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        predict = torch.argmax(model(x_test).softmax(dim=1), dim=1).squeeze()
        Q += (torch.mean((predict == torch.argmax(y_test, dim=1).squeeze()).float())).item()
        cnt += 1
Q /= cnt
print(f"Процент верных классификации на тестовой выборке: {round(Q*100, 3)}%")

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 15.53it/s]

Процент верных классификации на тестовой выборке: 93.616%


### Результат
* Получили довольно хорошую долю верно классицированных изображений, что на тренировочной выборке, что на тестовой

In [130]:
#Сохраним модель
torch.save(model.state_dict(), "model_dog_slassifier.tar")
print("Модель успешно сохранена!")

Модель успешно сохранена!
